- - -
## Data Loading & Augmentation
- - -

### MPC augmented dataset

In [ ]:
from typing import Any
import os
import numpy as np
import gym
import d4rl  # For auto registering the offline environments

from io_agent.runner.iterative import augment_mujoco_dataset, registered_envs
from io_agent.plant.mujoco import NominalMuJoCoEnv, AddXpositionWrapper
from io_agent.control.mpc import MuJoCoMPC
from io_agent.control.rmpc import MuJoCoRobustMPC

horizon = 10
rho = 1e6
env_name = "hopper-medium-v2"
model_fit_seed = 42

save_dir = "./{}_data/dataset".format(env_name.split("-")[0])
file_name = "mpc_{}_{}_{}".format(horizon, str(rho).replace(".", "-"), model_fit_seed)

# if not os.path.exists(os.path.join(save_dir, file_name)):
env = NominalMuJoCoEnv(
    AddXpositionWrapper(gym.make(env_name)),
    model_fit_seed=model_fit_seed)
env.params = env.fit_linear_model().params
expert = MuJoCoRobustMPC(
    action_size=env.action_size,
    state_size=env.state_size,
    noise_size=env.state_size,
    output_size=env.state_size,
    rho=rho,
    horizon=horizon,
)
expert.optimizer = expert.prepare_optimizer(params=env.nominal_model())

def shrink_dataset(dataset: Any, max_size: int) -> Any:
    keys = [
        "observations",
        "next_observations",
        "rewards",
        "terminals",
        "timeouts",
        "actions"
    ]
    for key in keys:
        dataset[key] = dataset[key][:max_size]
    return dataset

augment_mujoco_dataset(
    env=env,
    dataset=shrink_dataset(env.get_dataset(), 5000),
    expert_agent=expert,
    save_dir=save_dir,
    file_name=file_name,
)

- - -
## Iterative IO Controller Training
- - -

In [ ]:
import numpy as np
import torch
import multiprocessing

from io_agent.runner.iterative import run_iterative_io, IterativeIOArgs


n_cpu = multiprocessing.cpu_count()
n_trials = 20
general_seed = 42
seed_rng = np.random.default_rng(general_seed)
*trial_seeds, train_seed = seed_rng.integers(0, 2**30, n_trials + 1)
device = "cuda" if torch.cuda.is_available() else "cpu"


experiment_args = {
    # "Walker-IO-1e4": IterativeIOArgs(
    #     lr_exp_decay=0.9975,
    #     learning_rate=5e-2,
    #     n_batch=64,
    #     data_size=int(1e4),
    #     eval_epochs=tuple(range(0, 2601, 100)),
    #     env_name="walker",
    #     work_dir="./walker_data"),
    "Hopper-IO-5e3": IterativeIOArgs(
        lr_exp_decay=0.995,
        learning_rate=5e-2,
        n_batch=64,
        data_size=int(5e3),
        env_name="hopper",
        eval_epochs=tuple(range(0, 1001, 100)),
        work_dir="./hopper_data"),
    # "Cheetah-IO-1e4": IterativeIOArgs(
    #     lr_exp_decay=0.9925,
    #     learning_rate=5e-2,
    #     n_batch=64,
    #     data_size=int(1e4),
    #     eval_epochs=tuple(range(0, 2601, 100)),
    #     env_name="cheetah",
    #     work_dir="./cheetah_data"),
}

results = {}
for key, args in experiment_args.items():
    costs, epoch_losses, step_losses, iterative_io_agent = run_iterative_io(
        args=args,
        seed=train_seed,
        trial_seeds=trial_seeds,
        data_path="dataset/mpc_10_1000000-0_42",
        name=key,
        device=device,
        env_kwargs=dict(add_x_pos=True),
        verbose=True)
    results[key] = (costs, epoch_losses, step_losses)

- - -
## Visualizing the Training
- - -

In [ ]:
# Smoothing with last n steps
from collections import deque
from itertools import chain

from io_agent.plant.mujoco import HopperEnv


queue = deque(maxlen=5)
env = HopperEnv()

smooth_scores = {}
for name, scores in results.items():
    smooth_scores[name] = {}
    for key, values in scores[0].items():
        queue.append(values)
        smooth_scores[name][key] = list(map(lambda x: env.env.get_normalized_score(x) * 100, chain(*queue)))


In [ ]:
import numpy as np
from collections import defaultdict

from io_agent.plotter import tube_figure_plt


fig, axes = tube_figure_plt(
    cost_data=smooth_scores,
    title=f"",
    log_xaxis=False,
    log_yaxis=False,
    x_label="epoch",
    y_label="episodic score (%)",
    percentiles=(20, 80)
)

fig, axes = tube_figure_plt(
    cost_data={key: {index + 1: value for index,
                     value in enumerate(value[1])} for key, value in results.items()},
    title=f"",
    log_xaxis=True,
    log_yaxis=True,
    x_label="epoch",
    y_label="sub loss",
    percentiles=(20, 80)
)

fig, axes = tube_figure_plt(
    cost_data={key: {index + 1: value for index,
                     value in enumerate(value[2])} for key, value in results.items()},
    title=f"",
    log_xaxis=True,
    log_yaxis=True,
    x_label="gradient step",
    y_label="batch sub loss",
    percentiles=(20, 80)
)